In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import re
import csv
import json
import pprint
import json
from watson_developer_cloud import NaturalLanguageUnderstandingV1
import watson_developer_cloud.natural_language_understanding.features.v1 as Features

In [2]:
filename = '/home/claire/Documents/jobs/milieu/milieu-opinion-extraction-engine/noumea-sept13.json'
with open(filename) as file:
    json_file = json.load(file)

In [3]:
parameters = {
  'text': '',
  'features': {
      'entities': {
        'emotion': True,
        'sentiment': True,
        'limit' : 2
      },
      'sentiment': {
      'document': True
      },
      'keywords': {
        'emotion': True,
        'sentiment': True,
        'limit': 2
      },
      "concepts": {
      "limit": 2
      }
    },
    "language": "fr"
}


In [4]:
comments = []
analysis = []

natural_language_understanding = NaturalLanguageUnderstandingV1(
  username="ed5feef2-992f-4bce-bcf6-84fbbfe3ee4c",
  password="KSrLtnB5kamx",
  version="2017-02-27")

def parse_columns(json_file):
    for info in json_file:
        answers = info['answers']
        for y in answers:
            questionNumber = y['question']['question_number']
            questionBody = y['question']['body']
            textAnswer = y['body']
            if textAnswer:
                comments.append({'participant_id': info['id'], 
                                 'age': info['age'], 
                                 'neighbourhood': info['neighboorhood'],
                                 'lives_in_noumea': info['lives_in_noumea'],
                                 'months_lived_in_noumea': info['months_lived_in_noumea'],
                                 'email': info['email'],
                                 'enable_receive_emails': info['enable_receive_emails'],
                                 'question_number': questionNumber,
                                 'question_body': questionBody,
                                 'text': textAnswer})
    return call_watson(comments)
            
def call_watson(comments):
    for comment in comments:
        text = comment['text']
        if not (text.isdigit()) and len(comment['text']) > 5:
            print("Analyzing text...", text)
            response = natural_language_understanding.analyze(
              text=comment['text'],
              features=[
                Features.Entities(
                    emotion=True,
                    sentiment=True,
                    limit=2
                ),
                Features.Keywords(
                    emotion=True,
                    sentiment=True,
                    limit=2
                ),
                Features.Sentiment(
                    document=True
                ),
                Features.Categories()
              ],
              language="fr"
            )
            print(response)
            comment['watson_score'] = response['sentiment']['document']['score']
            comment['watson_keywords'] = response['keywords']
            comment['watson_entities'] = response['entities']
            comment['watson_categories'] = response['categories']
            analysis.append(comment)
        else:
            analysis.append(comment)
    return analysis
            

parse_columns(json_file)
pp = pprint.PrettyPrinter(indent=4)
# pp.pprint(analysis)

Analyzing text... il faudra mettre en place un service de sécurité, et que cela ne devienne pas un endroit à clochard. J'espère qu'il y aura des services de restauration pour se restaurer le midi.
{'keywords': [], 'categories': [{'label': '/technology and computing/computer security/network security', 'score': 0.624202}, {'label': '/food and drink/beverages/non alcoholic beverages/coffee and tea', 'score': 0.594835}, {'label': '/technology and computing/computer security', 'score': 0.328334}], 'entities': [], 'usage': {'features': 4, 'text_characters': 178, 'text_units': 1}, 'sentiment': {'document': {'label': 'neutral', 'score': 0.0}}, 'language': 'fr'}
Analyzing text... j'espère que les gares des bus ne ressembleront pas à celle de montravel
{'keywords': [], 'categories': [{'label': '/travel/transports/public transport', 'score': 0.999984}, {'label': '/automotive and vehicles/cars', 'score': 0.0038765}, {'label': '/travel/specialty travel/sightseeing tours', 'score': 0.00130261}], 'e

{'keywords': [{'relevance': 0.964844, 'text': 'espaces verts'}, {'relevance': 0.911033, 'text': 'fois culturel'}], 'categories': [{'label': '/business and industrial', 'score': 0.298661}, {'label': '/food and drink', 'score': 0.163148}, {'label': '/art and entertainment', 'score': 0.159241}], 'entities': [], 'usage': {'features': 4, 'text_characters': 177, 'text_units': 1}, 'sentiment': {'document': {'label': 'neutral', 'score': 0.0}}, 'language': 'fr'}
Analyzing text... Un lieu de loisirs et végétalisé si possible pour compenser les rejets de la SLN juste en face (un poumon vert)
{'keywords': [{'relevance': 0.930719, 'text': 'un poumon vert'}, {'relevance': 0.781994, 'text': 'SLN juste'}], 'categories': [{'label': '/home and garden/home improvement and repair/roofing', 'score': 0.442743}, {'label': '/science/medicine/surgery/transplants', 'score': 0.403878}, {'label': '/business and industrial/green solutions', 'score': 0.220521}], 'entities': [], 'usage': {'features': 4, 'text_charac

{'keywords': [{'relevance': 0.965612, 'text': 'voie express faisable'}], 'categories': [{'label': '/automotive and vehicles/vehicle rental', 'score': 0.419379}, {'label': '/art and entertainment/books and literature/mythology', 'score': 0.201109}, {'label': '/business and industrial', 'score': 0.160664}], 'entities': [], 'usage': {'features': 4, 'text_characters': 97, 'text_units': 1}, 'sentiment': {'document': {'label': 'positive', 'score': 0.668898}}, 'language': 'fr'}
Analyzing text... Manque emplacement toilettes publique. Très beau projet d'aménagement mais quel sera le coût de tout cela?
{'keywords': [{'relevance': 0.925534, 'text': 'Très beau projet'}], 'categories': [{'label': '/business and industrial/business operations/management/project management', 'score': 0.707111}, {'label': '/home and garden/bed and bath/bathroom', 'score': 0.474678}, {'label': '/home and garden/bed and bath/bedroom', 'score': 0.133704}], 'entities': [], 'usage': {'features': 4, 'text_characters': 106,

{'keywords': [{'relevance': 0.908229, 'text': 'artistiques= future zone'}, {'relevance': 0.803379, 'text': 'commerces envisagés'}], 'categories': [{'label': '/family and parenting', 'score': 0.17395}, {'label': '/home and garden', 'score': 0.150507}, {'label': '/home and garden/bed and bath/bathroom', 'score': 0.0987788}], 'entities': [{'type': 'Location', 'count': 1, 'sentiment': {'score': 0.0}, 'relevance': 0.978347, 'disambiguation': {'subtype': ['Country']}, 'text': 'RDC'}], 'usage': {'features': 4, 'text_characters': 656, 'text_units': 1}, 'sentiment': {'document': {'label': 'positive', 'score': 0.512233}}, 'language': 'fr'}
Analyzing text... Trop d'usages différents pour un même espace. Sensation "fourre tout": ligne directrice difficile à comprendre.
{'keywords': [{'relevance': 0.931615, 'text': 'directrice difficile'}], 'categories': [{'label': '/business and industrial/aerospace and defense/space technology', 'score': 0.668893}, {'label': '/hobbies and interests/needlework', '

{'keywords': [{'relevance': 0.90561, 'text': 'Un parc'}, {'relevance': 0.905603, 'text': 'dégradations....des bancs'}], 'categories': [{'label': '/family and parenting/children', 'score': 0.280392}, {'label': '/home and garden', 'score': 0.164997}, {'label': '/food and drink/food/grains and pasta', 'score': 0.129546}], 'entities': [], 'usage': {'features': 4, 'text_characters': 322, 'text_units': 1}, 'sentiment': {'document': {'label': 'neutral', 'score': 0.0}}, 'language': 'fr'}
Analyzing text... pas de goudrons....nature nature et nature
{'keywords': [{'relevance': 0.966712, 'text': 'goudrons....nature nature'}], 'categories': [], 'entities': [], 'usage': {'features': 4, 'text_characters': 42, 'text_units': 1}, 'sentiment': {'document': {'label': 'neutral', 'score': 0.0}}, 'language': 'fr'}
Analyzing text... pas de voiture, de vélo.
Un endroit pour les piétons, les enfants et les poussettes.
{'keywords': [{'relevance': 0.99943, 'text': 'Un endroit'}], 'categories': [{'label': '/sport

...

{'keywords': [{'relevance': 0.966158, 'text': 'mercantil pur'}], 'categories': [{'label': '/art and entertainment/theatre/theatre companies', 'score': 0.566052}, {'label': '/sports/gymnastics', 'score': 0.453922}, {'label': '/art and entertainment/dance/modern dance', 'score': 0.219662}], 'entities': [], 'usage': {'features': 4, 'text_characters': 125, 'text_units': 1}, 'sentiment': {'document': {'label': 'neutral', 'score': 0.0}}, 'language': 'fr'}
Analyzing text... A première vue, pas assez de places de parking...
{'keywords': [], 'categories': [{'label': '/travel/transports/air travel/airports', 'score': 0.574656}, {'label': '/home and garden/laundry', 'score': 0.560453}, {'label': '/travel/transports/public transport', 'score': 0.235988}], 'entities': [], 'usage': {'features': 4, 'text_characters': 49, 'text_units': 1}, 'sentiment': {'document': {'label': 'negative', 'score': -0.393619}}, 'language': 'fr'}
Analyzing text... Les hamacs seraient de bons dortoirs pour les alcolisés
{'

{'keywords': [], 'categories': [{'label': '/home and garden', 'score': 0.709483}, {'label': '/travel/tourist destinations', 'score': 0.106468}, {'label': '/travel/hotels', 'score': 0.105385}], 'entities': [], 'usage': {'features': 4, 'text_characters': 192, 'text_units': 1}, 'sentiment': {'document': {'label': 'neutral', 'score': 0.0}}, 'language': 'fr'}
Analyzing text... anticiper les aménagements au vu des dégradations/saccages par les usagers (tags très importants dans la ville, beaucoup de destruction de matériel) et du coût de maintenance (jets d'eau...)
{'keywords': [{'relevance': 0.980354, 'text': "jets d'eau"}], 'categories': [{'label': '/technology and computing', 'score': 0.205666}, {'label': '/travel/transports/air travel/airplanes', 'score': 0.187929}, {'label': '/business and industrial/chemicals industry/plastics and polymers', 'score': 0.141271}], 'entities': [], 'usage': {'features': 4, 'text_characters': 190, 'text_units': 1}, 'sentiment': {'document': {'label': 'negat

{'keywords': [{'relevance': 0.994408, 'text': 'Un projet ambitieux'}, {'relevance': 0.841905, 'text': 'sûrement utile'}], 'categories': [{'label': '/family and parenting/children', 'score': 0.462313}, {'label': '/finance/personal finance/financial planning/retirement and pension', 'score': 0.262611}, {'label': '/finance/personal finance/financial planning/estate planning', 'score': 0.255561}], 'entities': [{'type': 'GeographicFeature', 'sentiment': {'score': 0.0}, 'count': 1, 'relevance': 0.963296, 'text': 'du Pacifique'}, {'type': 'Location', 'count': 1, 'sentiment': {'score': 0.0}, 'relevance': 0.408444, 'disambiguation': {'subtype': ['City']}, 'text': 'Calédonie'}], 'usage': {'features': 4, 'text_characters': 1056, 'text_units': 1}, 'sentiment': {'document': {'label': 'neutral', 'score': 0.0}}, 'language': 'fr'}
Analyzing text... Pourquoi ne pas transformer l'ancien hôpital en gare d'échange, c'est l'entrée de la ville pour le Néobus, Car sud et Rai et ensuite c'est le bus Karuia qu

{'keywords': [{'relevance': 0.990075, 'text': "station d'épuration actuelle"}, {'relevance': 0.928839, 'text': 'Prévoir aménagement'}], 'categories': [{'label': '/travel/transports/sea travel/cruises', 'score': 0.684687}, {'label': '/finance/personal finance/financial planning/estate planning', 'score': 0.307898}, {'label': '/home and garden/gardening and landscaping/yard and patio', 'score': 0.275705}], 'entities': [{'type': 'Sport', 'sentiment': {'score': 0.0}, 'count': 1, 'relevance': 0.958279, 'text': 'piscine'}, {'type': 'Location', 'count': 1, 'sentiment': {'score': 0.431487}, 'relevance': 0.214716, 'disambiguation': {'subtype': ['Region']}, 'text': 'nord'}], 'usage': {'features': 4, 'text_characters': 963, 'text_units': 1}, 'sentiment': {'document': {'label': 'neutral', 'score': 0.0}}, 'language': 'fr'}
Analyzing text... - Plan difficilement lisible
- Idée intéressante d'aménager zone détente et verte côté bord de mer
- Convient de mutualiser les axes empruntés par ensemble des 

{'keywords': [{'relevance': 0.985874, 'text': 'Les aménagement proposés'}, {'relevance': 0.889641, 'text': 'bonnes idées...mais attention'}], 'categories': [{'label': '/society/unrest and war', 'score': 0.57672}, {'label': '/food and drink/health and lowfat cooking', 'score': 0.572586}, {'label': '/art and entertainment/visual art and design/painting', 'score': 0.5601}], 'entities': [{'type': 'Person', 'sentiment': {'score': 0.0}, 'count': 1, 'relevance': 0.0993096, 'text': 'Vigils'}], 'usage': {'features': 4, 'text_characters': 539, 'text_units': 1}, 'sentiment': {'document': {'label': 'neutral', 'score': 0.0}}, 'language': 'fr'}
Analyzing text... Mettre un (ou plusieurs !) cafés, salon de thé est une bonne idée. Mais pourquoi le(s) mettre(s) si loin de l'entrée ? 
{'keywords': [{'relevance': 0.9827, 'text': 'bonne idée'}], 'categories': [{'label': '/food and drink/beverages/non alcoholic beverages/coffee and tea', 'score': 0.699772}, {'label': '/science/social science/history/prehist

In [8]:
#json.dumps() method turns a Python data structure into JSON:
jsonData = json.dumps(analysis)
 
#Encode JSON data
with open('sept13.json', 'w') as f:
     json.dump(jsonData, f)

In [20]:
import re
import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')
pattern = re.compile(r'^\d+$')
pattern2 = re.compile(r'[NOM]')
def print_things(analysis):
    for thing in analysis:
        if not pattern.match(thing['text']):
            tags = tagger.tag_text(thing['text'])
            if pattern2.match(tags[0]):
                pp.pprint(tags)



print_things(analysis)
# pp.pprint(analysis)

[   'Multiplication\tNOM\tmultiplication',
    'des\tPRP:det\tdu',
    'espaces\tNOM\tespace',
    'verts\tADJ\tvert',
    'sur\tPRP\tsur',
    'tout\tPRO:IND\ttout',
    'le\tDET:ART\tle',
    'site\tNOM\tsite',
    "c'\tPRO:DEM\tce",
    'est\tVER:pres\têtre',
    'très\tADV\ttrès',
    'bien\tADV\tbien',
    'par\tPRP\tpar',
    'contre\tPRP\tcontre',
    'le\tDET:ART\tle',
    'skate\tNOM\tskate',
    'parc\tNOM\tparc',
    "c'\tPRO:DEM\tce",
    'est\tVER:pres\têtre',
    'moyen\tADJ\tmoyen']
[   'Manque\tADJ\tmanque',
    'emplacement\tNOM\templacement',
    'toilettes\tNOM\ttoilette',
    'publique\tADJ\tpublique',
    '.\tSENT\t.',
    'Très\tADV\ttrès',
    'beau\tADJ\tbeau',
    'projet\tNOM\tprojet',
    "d'\tPRP\tde",
    'aménagement\tNOM\taménagement',
    'mais\tKON\tmais',
    'quel\tPRO:REL\tquel',
    'sera\tVER:futu\têtre',
    'le\tDET:ART\tle',
    'coût\tNOM\tcoût',
    'de\tPRP\tde',
    'tout\tPRO:IND\ttout',
    'cela\tPRO:DEM\tcela',
    '?\tSENT\t?']
[   "N'\

    'le\tDET:ART\tle',
    'rond-point\tNOM\trond-point',
    'Berthelot\tNAM\tBerthelot',
    'c\tVER:futu\tc',
    'est\tVER:pres\têtre',
    'là\tADV\tlà',
    'plutôt\tADV\tplutôt',
    "l'\tDET:ART\tle",
    'entrée\tNOM\tentrée',
    'de\tPRP\tde',
    'la\tDET:ART\tle',
    'ville\tNOM\tville',
    'Vélib\tNAM\tVélib',
    '(\tPUN\t(',
    'lol\tNOM\tlol',
    ')\tPUN\t)',
    'qui\tPRO:REL\tqui',
    'va\tVER:pres\taller',
    's\tVER:pper\ts',
    'en\tPRP\ten',
    'servir\tVER:infi\tservir',
    ',\tPUN\t,',
    'ils\tPRO:PER\til',
    'disparaîtront\tVER:futu\tdisparaître',
    'bien\tADV\tbien',
    'vite\tADV\tvite']
[   'On\tPRO:PER\ton',
    'en\tPRO:PER\ten',
    'fait\tVER:pres\tfaire',
    'des\tPRP:det\tdu',
    'choses\tNOM\tchose',
    'sur\tPRP\tsur',
    'papier\tNOM\tpapier',
    '(\tPUN\t(',
    'c\tVER:simp\tc',
    'est\tVER:pres\têtre',
    'beau\tADJ\tbeau',
    'de\tPRP\tde',
    'rêver\tVER:infi\trêver',
    ')\tPUN\t)',
    'encore\tADV\tencore',
    'q

[   'Nouvelle\tADJ\tnouveau',
    'caserne\tNOM\tcaserne',
    'pour\tPRP\tpour',
    'les\tDET:ART\tle',
    'pompier\tNOM\tpompier',
    '!\tSENT\t!',
    'Nouveau\tADJ\tnouveau',
    'commissariat\tNOM\tcommissariat',
    'de\tPRP\tde',
    'police\tNOM\tpolice',
    '!\tSENT\t!',
    'Et\tKON\tet',
    'des\tPRP:det\tdu',
    'parkings\tNOM\tparking',
    'pour\tPRP\tpour',
    'accéder\tVER:infi\taccéder',
    'au\tPRP:det\tau',
    'centre\tNOM\tcentre',
    'ville\tNOM\tville',
    'à\tPRP\tà',
    'pieds\tNOM\tpied',
    '!\tSENT\t!']
[   'Oui\tINT\toui',
    'oui\tINT\toui',
    'de\tPRP\tde',
    'la\tDET:ART\tle',
    'végétation\tNOM\tvégétation',
    '.\tSENT\t.',
    '.\tSENT\t.',
    'pas\tADV\tpas',
    'trop\tADV\ttrop',
    'de\tPRP\tde',
    'batiment\tNOM\tbatiment',
    '!\tSENT\t!',
    '...\tPUN\t...',
    ':\tPUN\t:',
    ')\tPUN\t)']
[   'Non\tADV\tnon',
    'mais\tKON\tmais',
    'Néobus\tNAM\tNéobus',
    '.\tSENT\t.',
    '.\tSENT\t.',
    'ils\tPRO:PER\til'